## STORM: 연구를 위한 멀티 에이전트

### 개요

STORM(Synthesis of Topic Outline through Retrieval and Multi-perspective Question Asking)은 Stanford 대학에서 개발한 LLM 기반의 지식 큐레이션 시스템입니다. 이 시스템은 인터넷 리서치를 통해 Wikipedia 수준의 포괄적이고 체계적인 장문의 기사를 자동으로 생성하는 것을 목표로 합니다.

![](https://github.com/stanford-oval/storm/raw/main/assets/two_stages.jpg)

### 핵심 아키텍처

STORM은 두 단계의 파이프라인으로 구성됩니다:

1. **사전 작성 단계(Pre-writing Stage)**
   - 인터넷 기반 리서치를 수행하여 참고 자료 수집
   - 다양한 관점(perspective) 발견
   - 주제에 대한 개요(outline) 생성

2. **작성 단계(Writing Stage)**
   - 생성된 개요와 수집된 참고 자료를 활용
   - 인용(citation)이 포함된 전체 기사 작성

### 멀티 에이전트 접근법

STORM의 핵심은 **관점 기반 질문 생성(Perspective-Guided Question Asking)** 과 **시뮬레이션된 대화(Simulated Conversation)** 전략입니다:

- **다양한 관점 발견**: 유사한 주제의 기존 기사들을 조사하여 다양한 시각을 발견하고, 이를 질문 생성 과정에 활용
- **역할 기반 대화 시뮬레이션**: Wikipedia 작성자와 주제 전문가 간의 대화를 시뮬레이션
  - 작성자 에이전트: 다양한 관점에서 질문 제기
  - 전문가 에이전트: 인터넷 소스에 기반한 답변 제공
  - 이를 통해 이해도를 업데이트하고 후속 질문 생성

### Co-STORM: 협업 확장

Co-STORM은 STORM을 협업 기능으로 확장한 버전으로, 다음과 같은 멀티 에이전트 구성을 포함합니다:


- **LLM 전문가 에이전트**: 외부 소스에 기반한 답변 생성 및 후속 질문 제기
- **중재자 에이전트(Moderator)**: 발견된 정보에서 영감을 받은 사고를 자극하는 질문 생성
- **동적 마인드맵**: 정보를 계층적으로 정리하여 인간과 시스템 간의 공유 개념 공간 생성

![](https://github.com/stanford-oval/storm/raw/main/assets/co-storm-workflow.jpg)

### 주요 특징

- **포괄적 커버리지**: 다양한 관점에서 주제를 탐색하여 Wikipedia 수준의 광범위한 내용 생성
- **구조화된 정보**: 자동으로 생성된 개요를 통해 체계적으로 정보 조직
- **신뢰할 수 있는 출처**: 인터넷 소스에 기반하여 모든 정보에 인용 포함
- **평가 검증**: FreshWiki 데이터셋을 통한 평가에서 기존 방법 대비 조직성 25%, 커버리지 10% 향상

STORM은 복잡한 연구 작업을 자동화하고, 다양한 관점에서 정보를 종합하며, 신뢰할 수 있는 장문의 리포트를 생성하는 멀티 에이전트 시스템의 우수한 사례입니다.

---

- 참고 자료: https://wikidocs.net/270693
- 관련 논문: https://arxiv.org/abs/2402.14207
- GitHub 저장소: https://github.com/stanford-oval/storm

## 환경 설정

In [2]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 분석가 생성: Human-In-The-Loop

분석가 생성이 필요한 클래스를 정의합니다.

In [8]:
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field


class Analyst(BaseModel):
    """분석가 속성과 메타데이터를 정의"""

    affiliation: Annotated[str, Field(description="분석가의 주요 소속 기관")]
    name: Annotated[str, Field(description="분석가 이름")]
    role: Annotated[str, Field(description="주제 맥락에서의 분석가의 역할")]
    description: Annotated[
        str, Field(description="분석가의 관심사, 우려 사항 및 동기 설명")
    ]

    @property
    def persona(self) -> str:
        return (
            f"이름: {self.name}\n"
            "역할: {self.role}\n"
            "소속 기관: {self.affiliation}\n"
            "설명: {self.description}\n"
        )


class Perspectives(BaseModel):
    """분석가들의 집합"""

    analysts: Annotated[
        list[Analyst],
        Field(description="분석가들의 역할 및 소속 기관을 포함한 종합 목록"),
    ]


### 분석가 생성 상태 및 노드 정의

In [4]:
# 상태 정의
class GenerateAnalystsState(TypedDict):
    topic: Annotated[str, "연구 주제"]
    max_analysts: Annotated[int, "생성할 분석가의 최대 수"]
    human_analyst_feedback: Annotated[str, "휴먼 피드백"]
    analysts: Annotated[list[Analyst], "분석가 목록"]

In [5]:
# 분석가 생성 프롬프트
analyst_instructions = """AI 애널리스트 페르소나 세트를 생성하는 임무를 맡았습니다.

다음 지침을 주의 깊게 따르십시오:

1. 먼저 연구 주제를 검토하십시오:

{topic}

2. 애널리스트 생성 가이드로 제공된 선택적 편집 피드백을 검토하십시오:

{human_analyst_feedback}

3. 위 문서 및/또는 피드백을 바탕으로 가장 흥미로운 테마를 결정하십시오.

4. 상위 {max_analysts}개 테마를 선정하십시오.

5. 각 테마에 한 명의 애널리스트를 배정하십시오."""

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage


llm = init_chat_model("openai:gpt-4.1-mini")


# 분석가 생성 노드
def create_analysts(state: GenerateAnalystsState):
    """분석가 페르소나를 생성합니다."""

    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state.get("human_analyst_feedback", "")

    llm_with_structured = llm.with_structured_output(Perspectives)

    system_message = analyst_instructions.format(
        topic=topic,
        human_analyst_feedback=human_analyst_feedback,
        max_analysts=max_analysts,
    )

    response = llm_with_structured.invoke(
        [SystemMessage(content=system_message)]
        + [HumanMessage(content="Generate the set of analysts.")]
    )

    return {
        "analysts": response.analysts,
    }

In [10]:
create_analysts({"topic": "멀티 에이전트", "max_analysts": 3})

{'analysts': [Analyst(affiliation='KAIST 인공지능연구소', name='김지훈', role='멀티 에이전트 시스템 이론 연구원', description='멀티 에이전트 간 협력과 경쟁 모델을 연구하며, 분산 인공지능과 협업 알고리즘의 이론적 기반을 탐구하는 전문가.'),
  Analyst(affiliation='서울대학교 컴퓨터공학과', name='이수민', role='멀티 에이전트 강화학습 연구원', description='멀티 에이전트 환경에서의 강화 학습 알고리즘 개발 및 응용에 집중하며, 자율 에이전트 간 상호작용 최적화 연구를 수행.'),
  Analyst(affiliation='네이버 AI 연구소', name='박현우', role='멀티 에이전트 기반 실시간 시스템 개발자', description='실시간 멀티 에이전트 시스템 설계 및 구현 전문가로, 분산 시스템과 에이전트 커뮤니케이션 프로토콜 개발 경험 다수 보유.')]}

In [11]:
# 사용자 피드백 노드
def human_feedback(state: GenerateAnalystsState):
    """사용자 피드백을 받기 위한 중단점 노드"""
    pass

In [ ]:
def should_continue(state: GenerateAnalystsState):
    """워크플로우의 다음 노드를 결정합니다."""

    human_analyst_feedback = state.get("human_analyst_feedback", "")
    if human_analyst_feedback:
        return "create_analysts"
    return "___end___"